# importation des librairies

In [ ]:
import pandas as pd #matrice
import sklearn as sk #Machine Learning
import numpy as np

from datetime import datetime
import time

from math import *
import statistics

from joblib import dump, load

#from blog saladtomatonion.com
class Timer(object):
    def __enter__(self):
        self.start()
        return self
    
    def __exit__(self, *args, **kwargs):
        self.stop()
        
    def start(self):
        if hasattr(self, 'interval'):
            del self.interval
        self.start_time = time.time()
    
    def stop(self):
        if hasattr(self, 'start_time'):
            self.interval = time.time() - self.start_time
            del self.start_time
            
def formatDFsansOthers(df):
    #mise en minuscule
    df["Entité"] = df["Entité"].str.lower()
    df["Compte"] = df["Compte"].str.lower()
    df["Fournisseur identification"] = df["Fournisseur identification"].str.lower()
    df["Cd_Scat"] = df["Cd_Scat"].str.lower()
    df["Fournisseur"] = df["Fournisseur"].str.lower()
    df["Family Stream"] = df["Family Stream"].str.lower()
    df["Category Stream"] = df["Category Stream"].str.lower()
    df["Sub Category Stream"] = df["Sub Category Stream"].str.lower()
    
    #suppression des lignes non voulues pour l'apprentissage :
    #les others qui ne sont pas des "transactions fees"
    #EN COMMENTAIRE POUR L'APPLICATION AU RUN et POUR POUVOIR APPRENDRE LES OTHERS EGALEMENT
    indexNames = df[ (df['Family Stream'] == 'others') & (df['Category Stream'] != 'transaction fees')].index
    df.drop(indexNames , inplace=True)

    #remplis les vide avec un tiret "-"
    df.fillna("-", inplace=True)

    #aggrega
    #df["AcléMin"]=df["Entité"]+" ; "+df["Compte"]+" ; "+df["Fournisseur identification"]
    #df["AcléMinE"]=df["Entité"]+" ; "+df["Compte"]+" ; "+df["Fournisseur identification"]

    df["AcléEt"]=df["Entité"]+" ; "+df["Compte"]+" ; "+df["Fournisseur identification"]+" ; "+df["Cd_Scat"]+" ; "+df["Fournisseur"]
    #df["AcléEtSansE"]=df["Compte"]+" ; "+df["Fournisseur identification"]+" ; "+df["Cd_Scat"]+" ; "+df["Fournisseur"]
    #df["AcléTout"]=df["Entité"]+';'+df["CodePays"]+';'+df["Compte"]+';'+df["Sous_catégorie_de_dépenses"]+';'+df["Catégorie_de_dépenses"]+';'+df["Famille_de_dépenses"]+';'+df["Fournisseur"]+';'+df["Fournisseur identification"]

    df["Acible"]=df["Family Stream"]+" ; "+df["Category Stream"]+" ; "+df["Sub Category Stream"]
    
    return df

def formatDFsansOthersToLearn(df):
    #mise en minuscule
    df["Entité"] = df["Entité"].str.lower()
    df["Compte"] = df["Compte"].str.lower()
    df["Fournisseur identification"] = df["Fournisseur identification"].str.lower()
    df["Cd_Scat"] = df["Cd_Scat"].str.lower()
    df["Fournisseur"] = df["Fournisseur"].str.lower()
    #df["Family Stream"] = df["Family Stream"].str.lower()
    #df["Category Stream"] = df["Category Stream"].str.lower()
    #df["Sub Category Stream"] = df["Sub Category Stream"].str.lower()
    
    #suppression des lignes non voulues pour l'apprentissage :
    #les others qui ne sont pas des "transactions fees"
    #EN COMMENTAIRE POUR L'APPLICATION AU RUN et POUR POUVOIR APPRENDRE LES OTHERS EGALEMENT
    #indexNames = df[ (df['Family Stream'] == 'others') & (df['Category Stream'] != 'transaction fees')].index
    #df.drop(indexNames , inplace=True)

    #remplis les vide avec un tiret "-"
    df.fillna("-", inplace=True)

    #aggrega
    #df["AcléMin"]=df["Entité"]+" ; "+df["Compte"]+" ; "+df["Fournisseur identification"]
    #df["AcléMinE"]=df["Entité"]+" ; "+df["Compte"]+" ; "+df["Fournisseur identification"]

    df["AcléEt"]=df["Entité"]+" ; "+df["Compte"]+" ; "+df["Fournisseur identification"]+" ; "+df["Cd_Scat"]+" ; "+df["Fournisseur"]
    #df["AcléEtSansE"]=df["Compte"]+" ; "+df["Fournisseur identification"]+" ; "+df["Cd_Scat"]+" ; "+df["Fournisseur"]
    #df["AcléTout"]=df["Entité"]+';'+df["CodePays"]+';'+df["Compte"]+';'+df["Sous_catégorie_de_dépenses"]+';'+df["Catégorie_de_dépenses"]+';'+df["Famille_de_dépenses"]+';'+df["Fournisseur"]+';'+df["Fournisseur identification"]

    #df["Acible"]=df["Family Stream"]+" ; "+df["Category Stream"]+" ; "+df["Sub Category Stream"]
    
    return df

In [46]:
#fichier d'apprentissage/Learn
#utilise un fichier avec un seul onglet pour simplifier
xlsx = pd.ExcelFile('./Streaming file-.xlsx')
dfK = pd.read_excel(xlsx, "Historique Streams")
dfKToLearn = pd.read_excel(xlsx, "à Streamer")

#dfK = pd.read_excel('./Streaming file-.xlsx')

dfKhalil = dfK.copy()
dfKhalilToLearn = dfKToLearn.copy()

dfKhalil = formatDFsansOthers(dfKhalil)
dump(dfKhalil, 'dfKhalil.dump')

dfKhalilToLearn = formatDFsansOthersToLearn(dfKhalilToLearn)
dump(dfKhalilToLearn, 'dfKhalilToLearn.dump')

['dfKhalilToLearn.dump']

In [4]:
dfKhalil = load('dfKhalil.dump')

In [2]:
dfKhalilToLearn = load('dfKhalilToLearn.dump')

In [22]:
#fichier d'apprentissage/Learn
#utilise un fichier avec un seul onglet pour simplifier
dfKToLearn2 = pd.read_excel('./Lignes à enrichir.xlsx')

dfKhalilToLearn2 = dfKToLearn2.copy()

dfKhalilToLearn2 = formatDFsansOthersToLearn(dfKhalilToLearn2)
dump(dfKhalilToLearn2, 'dfKhalilToLearn2.dump')

['dfKhalilToLearn2.dump']

In [ ]:
dfKhalilToLearn2 = load('dfKhalilToLearn2.dump')

In [ ]:
#fichier d'apprentissage/Learn
#utilise un fichier avec un seul onglet pour simplifier
dfL = pd.read_excel('./Cube_Global_View_2020_FY_v0.94 - pour ML.xlsx')

#sinon utiliser qq chose comme suit qui permet de spécifier l'onglet
#dfL = pd.read_excel('./Cube_Global_View_2020_FY_v0.94 - pour ML.xlsx', "Enriched Data", index_col=None, na_values=["NA"])
#ou en 2 temps
#xlsx = pd.ExcelFile('./Cube_Global_View_2020_FY_v0.94 - pour ML.xlsx')
#dfL = pd.read_excel(xlsx, "Enriched Data")

#suppression des colonnes non nécessaires,
#ou pouvant induire des résultats faux
#et pour travailler sur une copie
df2020 = dfL.copy() #.drop(columns=["MntKDevise", "SommeDekEUR", "Annee", "Période", "Devise", "Country", "Region"])

#fichier d'application/Restitution
dfR = pd.read_excel('./Cube 2021 Q1 v0.92.xlsx')
#pour travailler sur une copie
df2021Q1 = dfR.copy()
df2021Q1=df2021Q1.rename(columns={'kEUR': 'SommeDekEUR'})

dfGT = pd.read_excel('./Cube 2018 (V0.9 du 25.01.21).xlsx')
#pour travailler sur une copie
df2018 = dfGT.copy()
df2018=df2018.rename(columns={'kEUR': 'SommeDekEUR'})

#append des 2 dataframe (2020Q4 et 2021Q1)
df1921 = df2020.append(df2021Q1, ignore_index=True)
#append des 3 dataframe (2020Q4 et 2021Q1 et 2018Q4 (via 2019 Q4))
df1821 = df1921.append(df2018, ignore_index=True)

df1821 = formatDFsansOthers(df1821)
df1921 = formatDFsansOthers(df1921)

dump(df1821, 'df1821.dump')

In [3]:
df1821 = load('df1821.dump')

# Machine Learning

In [5]:
#début ML
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier as DT
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.impute import KNNImputer as KNNI
from sklearn import svm
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn import neighbors
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import classification_report

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.neural_network import MLPClassifier

from nltk.corpus import stopwords

## préparation

In [6]:
class chaine():
    def __init__(self, val, score, temps, model):
        self.valeur = val
        self.score = score
        self.temps = temps
        self.suivant = None
        self.precedent = None
        self.model = model
    
    def setSuivant(self, chaineSuivante):
        self.suivant = chaineSuivante
        chaineSuivante.setPrecedent(self)
        
    def setPrecedent(self, chainePrecedente):
        self.precedent = chainePrecedente
    
    def placeAvant(self, nouvelleChainePrecedente):
        if self.precedent != None:
            self.precedent.setSuivant(nouvelleChainePrecedente)
        else:
            nouvelleChainePrecedente.precedent = None
            
        nouvelleChainePrecedente.setSuivant(self)
    
    def placeApres(self, nouvelleChaineSuivante):
        if self.suivant != None:
            nouvelleChaineSuivante.setSuivant(self.suivant)            
        else:
            nouvelleChaineSuivante.suivant = None
            
        self.setSuivant(nouvelleChaineSuivante)
    
def getMaillonScoreMax(maillon):
    maxS = -1
    placeC = 1
    while True:
        if maillon.score > maxS :
            maxS = maillon.score
            maxM = maillon
            place = placeC
        
        if maillon.suivant != None:
            maillon=maillon.suivant
            placeC = placeC + 1
        else:
            return maxM, place
            break

def newValAutourduMax(maillon):
    valC = maillon.valeur
    
    if maillon.precedent != None:
        valPrec = maillon.precedent.valeur
    else:
        valPrec = valC/10

    if maillon.suivant != None:
        valSuiv = maillon.suivant.valeur
    else:
        valSuiv = valC*10
    
    newVP = valC/sqrt(valC/valPrec)
    newVS = valC*sqrt(valSuiv/valC)
    #pour accélerer la recherche autour du point de départ
    #newVP = valC/sqrt(sqrt(valC/valPrec))
    #newVS = valC*sqrt(sqrt(valSuiv/valC))
    
    return (newVP, newVS)
    
def apprendData(valeurVarie, criterion, n_estimators, tol, solver, loss, class_weight, typeML, labelData, X_train_vectorized, X_test_vectorized, y_train, y_test):
    label = str(valeurVarie)
    
    if typeML=='LogisticRegression':
        model = LogisticRegression(solver=solver, class_weight=class_weight, tol=tol, C=valeurVarie, max_iter=1200)#, l1_ratio=0.1)
    elif typeML=='PassiveAggressiveClassifier':
        model = PassiveAggressiveClassifier(class_weight=class_weight, tol=tol, C=valeurVarie, max_iter=800)
    elif typeML=='SGDClassifier':
        model = SGDClassifier(class_weight=class_weight, loss=loss, tol=tol, alpha=valeurVarie, max_iter=400)#, l1_ratio=0.1)
    elif typeML=='LinearSVC':
        model = LinearSVC(class_weight=class_weight, loss=loss, tol=tol, C=valeurVarie, max_iter = 1200)#(max_iter = 1000, C=5.5)
    elif typeML=='RandomForest':
        model = RandomForestClassifier(class_weight=class_weight, n_estimators=n_estimators, n_jobs = 2, criterion=criterion, max_depth=valeurVarie)
    
    with Timer() as timer:
        modelC = model.fit(X_train_vectorized, y_train)
        #dicModel[typeML][labelData][label] = model.fit(X_train_vectorized, y_train)
    print(str(datetime.now())+" - valeur : "+str(valeurVarie)+" - fit en : "+str(timer.interval))

    #print('************ :'+str(X_test_vectorized.shape[0])+': ************')
# !!!!! à rétablir pour apprentissage    score = modelC.score(X_test_vectorized,y_test)
    #score = dicModel[typeML][labelData][label].score(X_test_vectorized,y_test)

# !!!!! à rétablir pour apprentissage    print(str(datetime.now())+" -     score : "+str(100*score)+" - score train : "+str(100*modelC.score(X_train_vectorized, y_train)))
    #print(str(datetime.now())+" -   score train : "+str(100*dicModel[typeML][labelData][label].score(X_train_vectorized, y_train)))
    
    score=1 # !!!!! à supprimer pour apprentissage multi
    
    return score, timer.interval, modelC

def testParam(param, criterion, n_estimators, tol, solver, loss, class_weight, typeML, \
              labelData, nomCols, X_train, X_test, y_train, y_test, etape):
    #for param in [valBase, 10*valBase, 100*valBase]:
    score, temps, model = apprendData(param, criterion, n_estimators, tol, solver, loss, class_weight,
                                      typeML, labelData, X_train, X_test, y_train, y_test)

    #if param == valBase:
# !!!!! à rétablir pour apprentissage    premier = chaine(param, score, temps, model)
# !!!!! à rétablir pour apprentissage    courant = premier
    #else:
    #    courant.setSuivant(chaine(param, score, temps))
    #    courant = courant.suivant
    
    oldValMax =-1
    oldScoreMax =-1
    
    if etape == 1:
        maxVal = 0
    elif etape == 2:
        maxVal = 2
    else:
        maxVal = 4
    
    maxVal = 0 # !!!!! à enlever pour apprendre du multi
    #for i in range(4, (etape+1)*(etape+1)):
    for i in range(0, maxVal):
        print(i)
        maxMaillon, place = getMaillonScoreMax(premier)
        (nVP, nVS) = newValAutourduMax(maxMaillon)
        
        scoreC = maxMaillon.score
        if oldScoreMax < scoreC:
            oldScoreMax = scoreC
            print("nouveau Max !! : "+ str(100*scoreC))
        
        print(str(datetime.now().strftime("%Y-%m-%d_%H:%M:%S"))+" - max Score : "+str(100*maxMaillon.score)+" / val : "+str(maxMaillon.valeur))

        scoreP, tempsP, modelP = apprendData(nVP, criterion, n_estimators, tol, solver, loss, class_weight,
                                             typeML, labelData, X_train, X_test, y_train, y_test)
        scoreS, tempsS, modelS = apprendData(nVS, criterion, n_estimators, tol, solver, loss, class_weight,
                                             typeML, labelData, X_train, X_test, y_train, y_test)

        maxMaillon.placeAvant(chaine(nVP,scoreP, tempsP, modelP))
        maxMaillon.placeApres(chaine(nVS,scoreS, tempsS, modelS))

        if place == 1:
            premier = maxMaillon.precedent
    
# !!!!! à rétablir pour apprentissage    maxMaillon, place = getMaillonScoreMax(premier)
# !!!!! à rétablir pour apprentissage    print()
# !!!!! à rétablir pour apprentissage    print("Max  : "+str(100*maxMaillon.score))
    
    #dump(maxMaillon.model, labelData+'_'+nomCols+'_'+typeML+'_best_'+solver+'_'+criterion+'_'+loss+'_'+str(maxMaillon.valeur)+'_'+str(100*maxMaillon.score)+'.modele')
    
    return param, 100*score, temps, model # !!!!! à enlever pour apprendre du multi et rétablir la ligne suivante
    #return maxMaillon.valeur, 100*maxMaillon.score, maxMaillon.temps, maxMaillon.model    

In [7]:
def transformXY(vect_V, X_test, y_test, label, nomCols, filtre, etape):
    if filtre != '':
        X_testFiltre = X_test[X_test[label+str(etape-1)] == filtre]
        y_testF = X_test[X_test[label+str(etape-1)] == filtre]
        y_testF = y_testF['Etape'+str(etape)]
    else:
        X_testFiltre = X_test
        y_testF = y_test
    
    nbrLignesFiltre = len(X_testFiltre)
    #print('nbrLignesFiltre : '+str(nbrLignesFiltre))
    if nbrLignesFiltre > 0:
        X_Te = X_testFiltre[nomCols]
        X_test_vectorized = vect_V.transform(X_Te)
        OK = True
    else:
        X_test_vectorized=''
        OK = False

    return X_test_vectorized, y_testF, OK
    

def testTout(label, nomCols, vect_V, X_train_vectorized, X_test, y_train, y_test, filtre, etape):
    paramBaseLR = 60
    paramBasePA = 0.5
    paramBaseSGD = 1e-8
    paramBaseLSVC = 40
    paramBaseRF = 500
    
    LogistiqueRegression_OK = True
    PassiveAggressive_OK = True
    SGD_OK = True
    LinearSVC_OK = True
    RandomForest_OK = False
    
    recap=[]
    recapModels={}
    loss = "-"
    tol = 1e-4
    class_weight = None
    criterion = "-"
    solver = "-"
    n_estimators = "-"
    
    contenuOK = True # !!!!!
    
    if LogistiqueRegression_OK:
        print('= = = = = = = = = = = = = = = >> Logistic Regression')
        #for solver in ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']:
        for solver in ['sag']: #'liblinear', , 'saga']:
            #print()
            print("===> solver : "+str(solver))
            #for class_weight in ['balanced', None]:
            #    print()
            #    print("===> class_weight : "+str(class_weight))
                #for tol in [1e-4]:#[1e-5, 1e-4, 1e-3]:
                #    print("===> tol : "+str(tol))
            if solver == 'liblinear':
                if etape == 1:
                    paramBaseLR = 125
                elif etape == 2:
                    if filtre[1]=='banking services':
                        paramBaseLR = 40
                    elif filtre[1]=='corporate services':
                        paramBaseLR = 176
                    elif filtre[1]=='professional services':
                        paramBaseLR = 54
                    elif filtre[1]=='technology':
                        paramBaseLR = 97
                elif etape == 3:
                    if filtre[1]=='corporate real estate':
                        paramBaseLR = 132
                    elif filtre[1]=='general services':
                        paramBaseLR = 59
                    elif filtre[1]=='travel':
                        paramBaseLR = 71
                    elif filtre[1]=='legal':
                        paramBaseLR = 37
                    elif filtre[1]=='marcom':
                        paramBaseLR = 82
                    elif filtre[1]=='recruitment fees':
                        paramBaseLR = 21
            else:
                if etape == 1:
                    paramBaseLR = 60
                elif etape == 2:
                    if filtre[1]=='banking services':
                        paramBaseLR = 19
                    elif filtre[1]=='corporate services':
                        paramBaseLR = 60
                    elif filtre[1]=='professional services':
                        paramBaseLR = 48
                    elif filtre[1]=='technology':
                        paramBaseLR = 30
                elif etape == 3:
                    if filtre[1]=='corporate real estate':
                        paramBaseLR = 31
                    elif filtre[1]=='general services':
                        paramBaseLR = 13
                    elif filtre[1]=='travel':
                        paramBaseLR = 27
                    elif filtre[1]=='legal':
                        paramBaseLR = 6
                    elif filtre[1]=='marcom':
                        paramBaseLR = 39
                    elif filtre[1]=='recruitment fees':
                        paramBaseLR = 18
            
            if False:
                X_test_vectorized, y_testF, contenuOK = transformXY(vect_V, X_test, y_test, 'LogisticRegression_'+solver+'_',
                                                         nomCols, filtre[1], etape)
            if contenuOK:
                maxVal, maxScore, maxTemps, maxModel = testParam(paramBaseLR, criterion, n_estimators, tol, solver, loss,
                                                                 class_weight, 'LogisticRegression', label, nomCols,
                                                                 X_train_vectorized, [], y_train, [], etape)
                recap.append([maxScore, maxVal, maxTemps, nomCols, 'LogisticRegression', 
                              tol, class_weight, solver, loss, filtre[1], etape])
                
                if etape > 1:
                    #print('vecteur etape precedente')
                    #print(X_test['LogisticRegression_'+solver+'_'+str(etape-1)])
                    #print()
                    #print('filtre cherché')
                    #print(filtre[1])
                    #print(filtre)
                    #print()
                    recapModels['LogisticRegression_'+solver+'_'+filtre[1]+'_'+str(etape)]=maxModel
                    if False:
                        valeurs = maxModel.predict(vect_V.transform(X_test[nomCols]))
                        X_test['LogisticRegression_'+solver+'_'+str(etape)] = \
                            [b if a == filtre[1] else c \
                             for a,b,c in zip(X_test['LogisticRegression_'+solver+'_'+str(etape-1)], valeurs, \
                                            X_test['LogisticRegression_'+solver+'_'+str(etape)])]
                    #print('vecteur construit à cette étape')
                    #print(X_test['LogisticRegression_'+solver+'_'+str(etape)])
                else:
                    recapModels['LogisticRegression_'+solver+'_'+str(etape)]=maxModel
                    if False:
                        X_test['LogisticRegression_'+solver+'_'+str(2)] = '-'
                        X_test['LogisticRegression_'+solver+'_'+str(3)] = '-'
            
    if PassiveAggressive_OK:
        #print()
        #print()
        print('= = = = = = = = = = = = = = = >> Passive Aggressive')
        solver="-"
        #for class_weight in ['balanced', None]:
        #    print()
        #    print("===> class_weight : "+str(class_weight))
            #for tol in [1e-4]:#[1e-4, 1e-3, 1e-2]:
            #    print("===> tol : "+str(tol))
        
        if etape == 2:
            if filtre[1]=='banking services':
                paramBasePA = 0.27
            elif filtre[1]=='corporate services':
                paramBasePA = 0.33
            elif filtre[1]=='professional services':
                paramBasePA = 0.76
            elif filtre[1]=='technology':
                paramBasePA = 0.31
        elif etape == 3:
            if filtre[1]=='corporate real estate':
                paramBasePA = 0.2
            elif filtre[1]=='general services':
                paramBasePA = 0.41
            elif filtre[1]=='travel':
                paramBasePA = 0.57
            elif filtre[1]=='legal':
                paramBasePA = 0.24
            elif filtre[1]=='marcom':
                paramBasePA = 0.22
            elif filtre[1]=='recruitment fees':
                paramBasePA = 0.09
        
        if False:
            X_test_vectorized, y_testF, contenuOK = transformXY(vect_V, X_test, y_test, 'PassiveAggressive_', 
                                                 nomCols, filtre[1], etape)
        if contenuOK:
            maxVal, maxScore, maxTemps, maxModel = testParam(paramBasePA, criterion, n_estimators, tol, solver, loss,
                                                             class_weight, 'PassiveAggressiveClassifier', label, nomCols,
                                                             X_train_vectorized, [], y_train, [], etape)
            recap.append([maxScore, maxVal, maxTemps, nomCols, 'PassiveAggressive', 
                          tol, class_weight, solver, loss, filtre[1], etape])
            #recapModels['PassiveAggressive'+'_'+filtre[1]+'_'+str(etape)]=maxModel

            if etape > 1:
                recapModels['PassiveAggressive_'+filtre[1]+'_'+str(etape)]=maxModel
                if False:
                    valeurs = maxModel.predict(vect_V.transform(X_test[nomCols]))
                    X_test['PassiveAggressive_'+str(etape)] = \
                        [b if a == filtre[1] else c \
                         for a,b,c in zip(X_test['PassiveAggressive_'+str(etape-1)], valeurs, \
                                        X_test['PassiveAggressive_'+str(etape)])]
            else:
                recapModels['PassiveAggressive'+'_'+str(etape)]=maxModel
                if False:
                    X_test['PassiveAggressive_'+str(2)] = '-'
                    X_test['PassiveAggressive_'+str(3)] = '-'
                
    if SGD_OK:
        #print()
        #print()
        print('= = = = = = = = = = = = = = = >> SGD')
        for loss in ['hinge', 'log', 'modified_huber']:#, 'squared_hinge', 'perceptron']:
            #print()
            print("===> loss : "+str(loss))
            #for class_weight in ['balanced', None]:
            #    print()
            #    print("===> class_weight : "+str(class_weight))
                #for tol in [1e-4]:#[1e-4, 1e-3, 1e-2]:
                #    print("===> tol : "+str(tol))
            if loss=='hinge':
                if etape == 1:
                    paramBaseSGD = 2e-8
                elif etape == 2:
                    if filtre[1]=='banking services':
                        paramBaseSGD = 6.3e-9
                    elif filtre[1]=='corporate services':
                        paramBaseSGD = 2.8e-8
                    elif filtre[1]=='professional services':
                        paramBaseSGD = 1.5e-8
                    elif filtre[1]=='technology':
                        paramBaseSGD = 1.8e-8
                elif etape == 3:
                    if filtre[1]=='corporate real estate':
                        paramBaseSGD = 2e-8
                    elif filtre[1]=='general services':
                        paramBaseSGD = 1.6e-8
                    elif filtre[1]=='travel':
                        paramBaseSGD = 5.4e-8
                    elif filtre[1]=='legal':
                        paramBaseSGD = 2e-9
                    elif filtre[1]=='marcom':
                        paramBaseSGD = 1.6e-8
                    elif filtre[1]=='recruitment fees':
                        paramBaseSGD = 7e-9
            elif loss=='log':
                if etape == 1:
                    paramBaseSGD = 1e-8
                elif etape == 2:
                    if filtre[1]=='banking services':
                        paramBaseSGD = 6.7e-9
                    elif filtre[1]=='corporate services':
                        paramBaseSGD = 8.4e-9
                    elif filtre[1]=='professional services':
                        paramBaseSGD = 1e-8
                    elif filtre[1]=='technology':
                        paramBaseSGD = 8e-9
                elif etape == 3:
                    if filtre[1]=='corporate real estate':
                        paramBaseSGD = 1.4e-8
                    elif filtre[1]=='general services':
                        paramBaseSGD = 1e-8
                    elif filtre[1]=='travel':
                        paramBaseSGD = 1.1e-8
                    elif filtre[1]=='legal':
                        paramBaseSGD = 2.1e-9
                    elif filtre[1]=='marcom':
                        paramBaseSGD = 1.1e-8
                    elif filtre[1]=='recruitment fees':
                        paramBaseSGD = 5.5e-9
            elif loss=='modified_huber':
                if etape == 1:
                    paramBaseSGD = 1e-7
                elif etape == 2:
                    if filtre[1]=='banking services':
                        paramBaseSGD = 4.5e-8
                    elif filtre[1]=='corporate services':
                        paramBaseSGD = 1.75e-7
                    elif filtre[1]=='professional services':
                        paramBaseSGD = 1.46e-7
                    elif filtre[1]=='technology':
                        paramBaseSGD = 9.2e-8
                elif etape == 3:
                    if filtre[1]=='corporate real estate':
                        paramBaseSGD = 8.7e-8
                    elif filtre[1]=='general services':
                        paramBaseSGD = 1.1e-7
                    elif filtre[1]=='travel':
                        paramBaseSGD = 1.5e-7
                    elif filtre[1]=='legal':
                        paramBaseSGD = 1e-8
                    elif filtre[1]=='marcom':
                        paramBaseSGD = 1.5e-7
                    elif filtre[1]=='recruitment fees':
                        paramBaseSGD = 6.3e-8
        
            if False:
                X_test_vectorized, y_testF, contenuOK = transformXY(vect_V, X_test, y_test, 'SGD_'+loss+'_', 
                                                     nomCols, filtre[1], etape)
            if contenuOK:
                maxVal, maxScore, maxTemps, maxModel = testParam(paramBaseSGD, criterion, n_estimators, tol, solver,
                                                                 loss, class_weight, 'SGDClassifier', label, nomCols,
                                                                 X_train_vectorized, [], y_train, [], etape)
                recap.append([maxScore, maxVal, maxTemps, nomCols, 'SGD', tol, class_weight, 
                              solver, loss, filtre[1], etape])
                #recapModels['SGD_'+loss+'_'+filtre[1]+'_'+str(etape)]=maxModel

                if etape > 1:
                    recapModels['SGD_'+loss+'_'+filtre[1]+'_'+str(etape)]=maxModel
                    if False:
                        valeurs = maxModel.predict(vect_V.transform(X_test[nomCols]))
                        X_test['SGD_'+loss+'_'+str(etape)] = \
                            [b if a == filtre[1] else c \
                             for a,b,c in zip(X_test['SGD_'+loss+'_'+str(etape-1)], valeurs, \
                                            X_test['SGD_'+loss+'_'+str(etape)])]
                else:
                    recapModels['SGD_'+loss+'_'+str(etape)]=maxModel
                    if False:
                        X_test['SGD_'+loss+'_'+str(2)] = '-'
                        X_test['SGD_'+loss+'_'+str(3)] = '-'
             
    if LinearSVC_OK:
        #print()
        #print()
        print('= = = = = = = = = = = = = = = >> Linear SVC')
        for loss in ['hinge', 'squared_hinge']:
            #print()
            print("===> loss : "+str(loss))
            #for class_weight in ['balanced', None]:
            #    print()
            #    print("===> class_weight : "+str(class_weight))
                #for tol in [1e-4]:#[1e-5, 1e-4, 1e-3]:
                #    print("===> tol : "+str(tol))
            if loss=='hinge':
                if etape == 1:
                    paramBaseLSVC = 40
                elif etape == 2:
                    if filtre[1]=='banking services':
                        paramBaseLSVC = 21.7
                    elif filtre[1]=='corporate services':
                        paramBaseLSVC = 32
                    elif filtre[1]=='professional services':
                        paramBaseLSVC = 46.2
                    elif filtre[1]=='technology':
                        paramBaseLSVC = 18
                elif etape == 3:
                    if filtre[1]=='corporate real estate':
                        paramBaseLSVC = 36.3
                    elif filtre[1]=='general services':
                        paramBaseLSVC = 23.3
                    elif filtre[1]=='travel':
                        paramBaseLSVC = 52
                    elif filtre[1]=='legal':
                        paramBaseLSVC = 6.6
                    elif filtre[1]=='marcom':
                        paramBaseLSVC = 35
                    elif filtre[1]=='recruitment fees':
                        paramBaseLSVC = 5.4
            elif loss=='squared_hinge':
                if etape == 1:
                    paramBaseLSVC = 7.5
                elif etape == 2:
                    if filtre[1]=='banking services':
                        paramBaseLSVC = 2.4
                    elif filtre[1]=='corporate services':
                        paramBaseLSVC = 3.2
                    elif filtre[1]=='professional services':
                        paramBaseLSVC = 6.5
                    elif filtre[1]=='technology':
                        paramBaseLSVC = 5.5
                elif etape == 3:
                    if filtre[1]=='corporate real estate':
                        paramBaseLSVC = 3.4
                    elif filtre[1]=='general services':
                        paramBaseLSVC = 5.9
                    elif filtre[1]=='travel':
                        paramBaseLSVC = 5.3
                    elif filtre[1]=='legal':
                        paramBaseLSVC = 1.2
                    elif filtre[1]=='marcom':
                        paramBaseLSVC = 5.1
                    elif filtre[1]=='recruitment fees':
                        paramBaseLSVC = 1.9
        
            if False:
                X_test_vectorized, y_testF, contenuOK = transformXY(vect_V, X_test, y_test, 'LinearSVC_'+loss+'_', 
                                                     nomCols, filtre[1], etape)
            if contenuOK:
                maxVal, maxScore, maxTemps, maxModel = testParam(paramBaseLSVC, criterion, n_estimators, tol, solver,
                                                                 loss, class_weight, 'LinearSVC', label, nomCols,
                                                                 X_train_vectorized, [], y_train, [], etape)
                recap.append([maxScore, maxVal, maxTemps, nomCols, 'LinearSVC', tol, class_weight, 
                              solver, loss, filtre[1], etape])
                #recapModels['LinearSVC_'+loss+'_'+filtre[1]+'_'+str(etape)]=maxModel

                if etape > 1:
                    recapModels['LinearSVC_'+loss+'_'+filtre[1]+'_'+str(etape)]=maxModel
                    if False:
                        valeurs = maxModel.predict(vect_V.transform(X_test[nomCols]))
                        X_test['LinearSVC_'+loss+'_'+str(etape)] = \
                            [b if a == filtre[1] else c \
                             for a,b,c in zip(X_test['LinearSVC_'+loss+'_'+str(etape-1)], valeurs, \
                                            X_test['LinearSVC_'+loss+'_'+str(etape)])]
                else:
                    recapModels['LinearSVC_'+loss+'_'+str(etape)]=maxModel
                    if False:
                        X_test['LinearSVC_'+loss+'_'+str(2)] = '-'
                        X_test['LinearSVC_'+loss+'_'+str(3)] = '-'
    
    if RandomForest_OK:
        print()
        print()
        print('= = = = = = = = = = = = = = = >> Random Forest')
        for criterion in ['gini', 'entropy']:
            print()
            print("===> criterion : "+str(criterion))
            for n_estimators in [100]: #[50, 100, 150, 200]:
                print()
                print("===> n_estimators : "+str(n_estimators))
                #for tol in [1e-4]:#[1e-5, 1e-4, 1e-3]:
                #    print("===> tol : "+str(tol))
        
                if False:
                    X_test_vectorized, y_testF, contenuOK = transformXY(vect_V, X_test, y_test, 'RandomForest_'+criterion+'_', 
                                                         nomCols, filtre[1], etape)
                if contenuOK:
                    maxVal, maxScore, maxTemps, maxModel = testParam(paramBaseRF, criterion, n_estimators, tol,
                                                                     solver, loss, class_weight, 'RandomForest',
                                                                     label, nomCols, X_train_vectorized, [],
                                                                     y_train, [], etape)
                    recap.append([maxScore, maxVal, maxTemps, nomCols, 'RandomForest',
                                  tol, class_weight, solver, loss, filtre[1], etape])
                    #recapModels['RandomForest_'+criterion+'_'+filtre[1]+'_'+str(etape)]=maxModel

                    if etape > 1:
                        recapModels['RandomForest_'+loss+'_'+filtre[1]+'_'+str(etape)]=maxModel
                        if False:
                            valeurs = maxModel.predict(vect_V.transform(X_test[nomCols]))
                            X_test['RandomForest_'+loss+'_'+str(etape)] = \
                                [b if a == filtre[1] else c \
                                 for a,b,c in zip(X_test['RandomForest_'+loss+'_'+str(etape-1)], valeurs, \
                                                X_test['RandomForest_'+loss+'_'+str(etape)])]
                    else:
                        recapModels['RandomForest_'+loss+'_'+str(etape)]=maxModel
                        if False:
                            X_test['RandomForest_'+loss+'_'+str(2)] = '-'
                            X_test['RandomForest_'+loss+'_'+str(3)] = '-'
               
    #dfRT = pd.DataFrame(recap, columns =['score', 'valeur', 'temps', 'nomCols', 'typeML', 'tol',
    #                                     'class_weight', 'solver', 'loss', 'filtre', 'etape'])
    #dfRT.to_excel(nomCols+'_'+filtre[1]+'_'+str(etape)+' - stats réduites.xlsx')
    
    return recap, recapModels, X_test, contenuOK

In [8]:
def testCol(X_Train, X_Test, y_Train, y_Test, vectorizer, filtre, etape):
    recap=[]
    
    for nomCols in ["AcléEt"]: #, "AcléEtSansE", "AcléMinE", "AcléMin"]:
        X_Tr=X_Train[nomCols]
        #X_Te=X_Test[nomCols]
        
        with Timer() as timer:
            vect_V = vectorizer.fit(X_Tr)
        print(str(datetime.now().strftime("%Y-%m-%d_%H:%M:%S"))+' - temps de traitement du fitting de '+nomCols+'+'+filtre[1]+': '+str(timer.interval))
        
        with Timer() as timer:
            X_TrV = vect_V.transform(X_Tr)
            #X_TeV = vect_V.transform(X_Te)
        print(str(datetime.now().strftime("%Y-%m-%d_%H:%M:%S"))+' - temps de traitement du transform : '+str(timer.interval))
        
        #print()
        print(str(datetime.now().strftime("%Y-%m-%d_%H:%M:%S"))+' !!! GO !!!')

        with Timer() as timerCol:
            #minirecap, recapModels = testTout('1821', nomCols, X_TrV, X_TeV, y_Train, y_Test, filtre, etape)
            minirecap, recapModels, retourXTest, contenuOK = testTout('1821', nomCols, vect_V, X_TrV, X_Test, \
                                                                      y_Train, y_Test, filtre, etape)
        #print()
        print(str(datetime.now().strftime("%Y-%m-%d_%H:%M:%S"))+' - temps de traitement de '+nomCols+': '+str(timerCol.interval))
        #print()
        
        recap = recap + minirecap
    
    if False:
        if etape > 1:
            dfR = retourXTest
        else:
            dfR = X_Test.copy()
            X_to_predict = vect_V.transform(dfR["AcléEt"])
            for label,model in recapModels.items():
                dfR[label] = model.predict(X_to_predict)
                #print(label)
    
    #dfRT = pd.DataFrame(recap, columns =['score', 'valeur', 'temps', 'nomCols', 'typeML', 'tol',
    #                                     'class_weight', 'solver', 'loss', 'filtre', 'etape'])
    #dfRT.to_excel(str(datetime.now().strftime("%Y.%m.%d_%H.%M.%S"))+'_'+filtre[1]+'_'+str(etape)+' - Stats globales.xlsx')
    
    #dfR.to_excel(str(datetime.now().strftime("%Y.%m.%d_%H.%M.%S"))+'_'+filtre[1]+'_'+str(etape)+' - sorties de Prediction.xlsx')
    
    return recap, [], recapModels, vect_V

def apprentissage(df, maxL, testSize, typeApprentissage, num_essai):
    print("---------------------------------------------------------------")
    print("-----------------------------DEBUT-----------------------------")
    print("---------------------------------------------------------------")
    print()
    print(str(datetime.now().strftime("%Y-%m-%d_%H:%M:%S"))+' - nombre de lignes : '+str(maxL))
    print(str(datetime.now().strftime("%Y-%m-%d_%H:%M:%S"))+' - taille du test : '+str(100*testSize)+'%')
    print()
    
    df['Etape1']=df['Family Stream']
    df['Etape2']=df['Category Stream']
    df['Etape3']=df['Sub Category Stream']
    
    if False: #pas de train_test_split en RUN
        with Timer() as timer:
            if maxL > 0:
                X_Train, X_Test, y_Train, y_Test = train_test_split(df[:maxL], df["Acible"][:maxL], test_size = testSize)
            else:
                X_Train, X_Test, y_Train, y_Test = train_test_split(df, df["Acible"], test_size = testSize)
        print(str(datetime.now().strftime("%Y-%m-%d_%H:%M:%S"))+' - temps de traitement du TTS : '+str(timer.interval))
    
    vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1,3),
                         min_df = 3, stop_words = 'english', sublinear_tf=True)
    
    if maxL > 0:
        df = df[:maxL]

    recapTout = []
    recapTousModels = []
    recapVecteur = []
    if typeApprentissage == 'tout':
        with Timer() as timerTout:
            #recap, dfR, recapModels = testCol(X_Train, X_Test, y_Train, y_Test, vectorizer, ['', ''], 1)
            recap, dfR, recapModels = testCol(df, [], df["Acible"], [], vectorizer, ['', ''], 1)
        print()
        print(str(datetime.now().strftime("%Y-%m-%d_%H:%M:%S"))+' - temps de traitement : '+str(timerTout.interval))
        recapTout = recapTout + recap
        recapTousModels.append(recapModels)
        
        dfRT = pd.DataFrame(recap, columns =['score', 'valeur', 'temps', 'nomCols', 'typeML', 'tol',
                                             'class_weight', 'solver', 'loss', 'filtre', 'etape'])
        dfRT.to_excel(str(datetime.now().strftime("%Y.%m.%d_%H.%M.%S"))+'_'+str(num_essai)+' - Stats globales.xlsx')

        dfR.to_excel(str(datetime.now().strftime("%Y.%m.%d_%H.%M.%S"))+'_'+str(num_essai)+' - sorties de Prediction.xlsx')
        dump(dfR, str(datetime.now().strftime("%Y.%m.%d_%H.%M.%S"))+'_'+str(num_essai)+' - sorties de Prediction.dump')
        
    else:
        if typeApprentissage == 'les 2':
            print('===> Etape 0 : apprend tout')
            with Timer() as timerTout:
                recap, dfR, recapModels, vecteur = testCol(df, [], df["Acible"], [], vectorizer, ['', ''], 0)
            #print()
            print(str(datetime.now().strftime("%Y-%m-%d_%H:%M:%S"))+' - temps de traitement : '+str(timerTout.interval))
            recapTout = recapTout + recap
            recapTousModels.append(recapModels)
            recapVecteur.append(vecteur)
            
        dfR = None
        print()
        print('===> Etape 1 : apprend par partie')
        with Timer() as timerTout:
            recap, dfRFamily, recapModels, vecteur = testCol(df, [], df["Family Stream"], [],
                                       vectorizer, ['', ''], 1)
        print()
        print(str(datetime.now().strftime("%Y-%m-%d_%H:%M:%S"))+' - temps de traitement : '+str(timerTout.interval))
        recapTout = recapTout + recap
        recapTousModels.append(recapModels)
        recapVecteur.append(vecteur)
        
        #X_TestFS = dfRFamily
        for familyStream in df.groupby(['Family Stream']).count().transpose().columns:
            print()
            print('===> Etape 2')
            print()
            print(familyStream)
            
            dfFS = df[df['Family Stream'] == familyStream]
            print(dfFS.shape)
            nbrCat = len(dfFS.groupby(['Category Stream']))
            print(nbrCat)
            if nbrCat > 1 and dfFS.shape[0] > 0:
                with Timer() as timerTout:
                    recap, X_TestFS, recapModels, vecteur = testCol(dfFS, [], dfFS["Category Stream"], [],
                                                 vectorizer, ['', familyStream], 2)
                #print(len(X_TestFS))
                print(str(datetime.now().strftime("%Y-%m-%d_%H:%M:%S"))+' - temps de traitement : '+str(timerTout.interval))
                recapTout = recapTout + recap
                recapTousModels.append(recapModels)
                recapVecteur.append(vecteur)

                #X_TestCS = X_TestFS
                for categoryStream in dfFS.groupby(['Category Stream']).count().transpose().columns:
                    print()
                    print('===> Etape 3')
                    print()
                    print(categoryStream)

                    dfCS = dfFS[dfFS['Category Stream'] == categoryStream]
                    print(dfCS.shape)
                    nbrSousCat = len(dfCS.groupby(['Sub Category Stream']))
                    print(nbrSousCat)
                    if nbrSousCat > 1 and dfCS.shape[0] > 0:
                        with Timer() as timerTout:
                            recap, X_TestCS, recapModels, vecteur = testCol(dfCS, [], dfCS["Sub Category Stream"], [],
                                                     vectorizer, [familyStream, categoryStream], 3)
                        #print(len(X_TestCS))
                        print(str(datetime.now().strftime("%Y-%m-%d_%H:%M:%S"))+' - temps de traitement : '+str(timerTout.interval))
                        recapTout = recapTout + recap
                        recapTousModels.append(recapModels)
                        recapVecteur.append(vecteur)
                        
        #dfR = X_TestCS
    
    return recapTout, [], recapTousModels, recapVecteur

# APPRENTISSAGE

In [36]:
dfToLearn800K = dfKhalil.copy()

with Timer() as timerTout:
    recapToutKhalil, dfRKhalil, recapTousModelsKhalil, recapVectRunKhalil = apprentissage(dfToLearn800K, 10000, 0, 'les 2', 0)
print()
print(str(datetime.now().strftime("%Y-%m-%d_%H:%M:%S"))+' - temps de traitement global : '+str(timerTout.interval))

dump(recapTousModelsKhalil, str(datetime.now().strftime("%Y.%m.%d_%H.%M.%S"))+'_'+'recapTousModelsKhalil.dump')
dump(recapVectRunKhalil, str(datetime.now().strftime("%Y.%m.%d_%H.%M.%S"))+'_'+'recapVectRunKhalil.dump')

dfToLearn1821 = df1821.copy()

with Timer() as timerTout:
    recapTout1821, dfR1821, recapTousModels1821, recapVectRun1821 = apprentissage(dfToLearn1821, 10000, 0, 'les 2', 0)
print()
print(str(datetime.now().strftime("%Y-%m-%d_%H:%M:%S"))+' - temps de traitement global : '+str(timerTout.interval))

dump(recapTousModels1821, str(datetime.now().strftime("%Y.%m.%d_%H.%M.%S"))+'_'+'recapTousModels1821.dump')
dump(recapVectRun1821, str(datetime.now().strftime("%Y.%m.%d_%H.%M.%S"))+'_'+'recapVectRun1821.dump')

---------------------------------------------------------------
-----------------------------DEBUT-----------------------------
---------------------------------------------------------------

2021-07-30_01:27:17 - nombre de lignes : 10000
2021-07-30_01:27:17 - taille du test : 0%

===> Etape 0 : apprend tout
2021-07-30_01:27:17 - temps de traitement du fitting de AcléEt+: 0.5543322563171387
2021-07-30_01:27:18 - temps de traitement du transform : 0.5474147796630859
2021-07-30_01:27:18 !!! GO !!!
= = = = = = = = = = = = = = = >> Logistic Regression
===> solver : sag
2021-07-30 01:27:22.258023 - valeur : 60 - fit en : 3.7143068313598633
= = = = = = = = = = = = = = = >> Passive Aggressive
2021-07-30 01:27:22.972546 - valeur : 0.5 - fit en : 0.7145228385925293
= = = = = = = = = = = = = = = >> SGD
===> loss : hinge
2021-07-30 01:27:23.345915 - valeur : 1e-08 - fit en : 0.37133169174194336
===> loss : log
2021-07-30 01:27:23.852106 - valeur : 1e-08 - fit en : 0.5041546821594238
===> loss : 

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


2021-07-30 01:27:26.657635 - valeur : 40 - fit en : 2.4617013931274414
===> loss : squared_hinge
2021-07-30 01:27:29.480632 - valeur : 40 - fit en : 2.8229968547821045
2021-07-30_01:27:29 - temps de traitement de AcléEt: 10.93691611289978
2021-07-30_01:27:29 - temps de traitement : 12.038663148880005

===> Etape 1 : apprend par partie
2021-07-30_01:27:30 - temps de traitement du fitting de AcléEt+: 0.637122631072998
2021-07-30_01:27:30 - temps de traitement du transform : 0.5487146377563477
2021-07-30_01:27:30 !!! GO !!!
= = = = = = = = = = = = = = = >> Logistic Regression
===> solver : sag
2021-07-30 01:27:31.853874 - valeur : 60 - fit en : 1.1874041557312012
= = = = = = = = = = = = = = = >> Passive Aggressive
2021-07-30 01:27:31.917984 - valeur : 0.5 - fit en : 0.06411027908325195
= = = = = = = = = = = = = = = >> SGD
===> loss : hinge
2021-07-30 01:27:31.953763 - valeur : 2e-08 - fit en : 0.03577876091003418
===> loss : log
2021-07-30 01:27:32.016260 - valeur : 1e-08 - fit en : 0.062

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


2021-07-30 01:27:32.602157 - valeur : 40 - fit en : 0.541919469833374
===> loss : squared_hinge
2021-07-30 01:27:32.695597 - valeur : 7.5 - fit en : 0.09233260154724121
2021-07-30_01:27:32 - temps de traitement de AcléEt: 2.0447568893432617

2021-07-30_01:27:32 - temps de traitement : 3.2305941581726074

===> Etape 2

banking services
(6441, 40)
7
2021-07-30_01:27:33 - temps de traitement du fitting de AcléEt+banking services: 0.3697183132171631
2021-07-30_01:27:33 - temps de traitement du transform : 0.319507360458374
2021-07-30_01:27:33 !!! GO !!!
= = = = = = = = = = = = = = = >> Logistic Regression
===> solver : sag
2021-07-30 01:27:34.436945 - valeur : 19 - fit en : 0.9830923080444336
= = = = = = = = = = = = = = = >> Passive Aggressive
2021-07-30 01:27:35.007461 - valeur : 0.27 - fit en : 0.5705163478851318
= = = = = = = = = = = = = = = >> SGD
===> loss : hinge
2021-07-30 01:27:35.208988 - valeur : 6.3e-09 - fit en : 0.20152640342712402
===> loss : log
2021-07-30 01:27:35.550150 - 

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


2021-07-30 01:27:37.554994 - valeur : 21.7 - fit en : 1.7607355117797852
===> loss : squared_hinge
2021-07-30 01:27:37.920248 - valeur : 2.4 - fit en : 0.36525416374206543
2021-07-30_01:27:37 - temps de traitement de AcléEt: 4.466395139694214
2021-07-30_01:27:37 - temps de traitement : 5.163732051849365

===> Etape 3

atm
(570, 40)
1

===> Etape 3

banking software & services
(3187, 40)
1

===> Etape 3

bs others
(133, 40)
1

===> Etape 3

cash counting
(184, 40)
1

===> Etape 3

cash transport
(662, 40)
1

===> Etape 3

cheques
(447, 40)
1

===> Etape 3

credit cards (incl processing)
(1258, 40)
1

===> Etape 2

bnp paribas group
(3559, 40)
1

2021-07-30_01:27:37 - temps de traitement global : 20.584659814834595


['2021.07.30_01.27.38_recapVectRunKhalil.dump']

# RESTITUTION

In [48]:
def preditStream(vecteur, X, models): #modelLR, modelPAC, modelSGD, modelLSVC):
    with Timer() as timer:
        X_to_predict = vecteur.transform(X)
    print(str(datetime.now())+' - temps de traitement du transform : '+str(timer.interval))
    
    predicted = []
    for label, model in models.items():
        with Timer() as timer:
            predicted.append(model.predict(X_to_predict))
        print(str(datetime.now())+' - temps de prediction de '+ label +' : '+str(timer.interval))
    
    return predicted

In [56]:
predictedKhalil = preditStream(recapVectRunKhalil[1], dfKhalilToLearn["AcléEt"], recapTousModelsKhalil[1])

2021-07-30 02:28:10.526932 - temps de traitement du transform : 0.7045066356658936


ValueError: X has 13647 features per sample; expecting 18896

In [37]:
recapTousModelsKhalil

[{'LogisticRegression_sag_0': LogisticRegression(C=60, max_iter=1200, solver='sag'),
  'PassiveAggressive_0': PassiveAggressiveClassifier(C=0.5, max_iter=800, tol=0.0001),
  'SGD_hinge_0': SGDClassifier(alpha=1e-08, max_iter=400, tol=0.0001),
  'SGD_log_0': SGDClassifier(alpha=1e-08, loss='log', max_iter=400, tol=0.0001),
  'SGD_modified_huber_0': SGDClassifier(alpha=1e-08, loss='modified_huber', max_iter=400, tol=0.0001),
  'LinearSVC_hinge_0': LinearSVC(C=40, loss='hinge', max_iter=1200),
  'LinearSVC_squared_hinge_0': LinearSVC(C=40, max_iter=1200)},
 {'LogisticRegression_sag_1': LogisticRegression(C=60, max_iter=1200, solver='sag'),
  'PassiveAggressive_1': PassiveAggressiveClassifier(C=0.5, max_iter=800, tol=0.0001),
  'SGD_hinge_1': SGDClassifier(alpha=2e-08, max_iter=400, tol=0.0001),
  'SGD_log_1': SGDClassifier(alpha=1e-08, loss='log', max_iter=400, tol=0.0001),
  'SGD_modified_huber_1': SGDClassifier(alpha=1e-07, loss='modified_huber', max_iter=400, tol=0.0001),
  'LinearSVC_

In [50]:
X_to_predict = recapVectRunKhalil[0].transform(dfKhalilToLearn["AcléEt"])

In [51]:
X_to_predict

<12750x13647 sparse matrix of type '<class 'numpy.float64'>'
	with 125645 stored elements in Compressed Sparse Row format>

In [52]:
recapTousModelsKhalil[0]

{'LogisticRegression_sag_0': LogisticRegression(C=60, max_iter=1200, solver='sag'),
 'PassiveAggressive_0': PassiveAggressiveClassifier(C=0.5, max_iter=800, tol=0.0001),
 'SGD_hinge_0': SGDClassifier(alpha=1e-08, max_iter=400, tol=0.0001),
 'SGD_log_0': SGDClassifier(alpha=1e-08, loss='log', max_iter=400, tol=0.0001),
 'SGD_modified_huber_0': SGDClassifier(alpha=1e-08, loss='modified_huber', max_iter=400, tol=0.0001),
 'LinearSVC_hinge_0': LinearSVC(C=40, loss='hinge', max_iter=1200),
 'LinearSVC_squared_hinge_0': LinearSVC(C=40, max_iter=1200)}

In [54]:
for label, model in recapTousModelsKhalil[0].items():
    print(label)
    print(model)

LogisticRegression_sag_0
LogisticRegression(C=60, max_iter=1200, solver='sag')
PassiveAggressive_0
PassiveAggressiveClassifier(C=0.5, max_iter=800, tol=0.0001)
SGD_hinge_0
SGDClassifier(alpha=1e-08, max_iter=400, tol=0.0001)
SGD_log_0
SGDClassifier(alpha=1e-08, loss='log', max_iter=400, tol=0.0001)
SGD_modified_huber_0
SGDClassifier(alpha=1e-08, loss='modified_huber', max_iter=400, tol=0.0001)
LinearSVC_hinge_0
LinearSVC(C=40, loss='hinge', max_iter=1200)
LinearSVC_squared_hinge_0
LinearSVC(C=40, max_iter=1200)


In [55]:
dftest=recapTousModelsKhalil[0]['LogisticRegression_sag_0'].predict(X_to_predict)

ValueError: X has 13647 features per sample; expecting 18896

In [58]:
dfKhalil["AcléEt"]

0         ukrsibbank ; means of payment / bank charges ;...
1         consorsbank ; operation expenses kordoba ; fis...
2         consorsbank ; operation expenses kordoba ; fis...
3         ukrsibbank ; means of payment / bank charges ;...
4         bddf - entreprises ; bank cards 24 months ; ge...
                                ...                        
823776    sevendays ; recurring telephony fees ; sinch s...
823777    sevendays ; recurring telephony fees ; tellus ...
823778    sevendays ; other it costs ; mybw office manag...
823779    sevendays ; misc. office supplies ; mybw offic...
823780    sevendays ; sundry building costs ; spicymind ...
Name: AcléEt, Length: 823781, dtype: object

# APPRENTISSAGE ET RESTITUTION SIMPLE

In [15]:
def definiVect(X):
    vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1,3),
                         min_df = 3, stop_words = 'english', sublinear_tf=True)
    
    print(str(datetime.now())+" - __start__")
    with Timer() as timer:
        vectRun = vectorizer.fit(X)
    print(str(datetime.now())+' - temps de traitement du fitting : '+str(timer.interval))
    
    dump(vectRun, 'vectRun.modele')
    
    return vectRun

def apprendModeles(X, y, labelDataApprises):
    vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1,3),
                         min_df = 3, stop_words = 'english', sublinear_tf=True)
    
    print(str(datetime.now())+" - __start__")
    with Timer() as timer:
        vectRun = vectorizer.fit(X)
    print(str(datetime.now())+' - temps de traitement du fitting : '+str(timer.interval))
    with Timer() as timer:
        X_RUN = vectRun.transform(X)
    print(str(datetime.now())+' - temps de traitement du transform : '+str(timer.interval))
    dump(vectRun, 'vectRun.modele')
    
    modelLRRun = LogisticRegression(solver='sag', C=60, max_iter=1200)
    modelPACRun = PassiveAggressiveClassifier(C=0.5, max_iter=800)
    modelSGDCRunH = SGDClassifier(alpha=2e-8, loss='hinge', max_iter=400)
    modelSGDCRunL = SGDClassifier(alpha=1e-8, loss='log', max_iter=400)
    modelSGDCRunMH = SGDClassifier(alpha=1e-7, loss='modified_huber', max_iter=400)
    modelLSVCRunH = LinearSVC(max_iter = 1200, loss='hinge', C=40)
    modelLSVCRunSH = LinearSVC(max_iter = 1200, loss='squared_hinge', C=7.5)
    
    with Timer() as timer:
        modelLRRun.fit(X_RUN, y)
    print(str(datetime.now())+' - temps de traitement du fit de modelLRRun : '+str(timer.interval))
    dump(modelLRRun, 'modeleLR_'+labelDataApprises+'.modele')
    
    with Timer() as timer:
        modelPACRun.fit(X_RUN, y)
    print(str(datetime.now())+' - temps de traitement du fit de modelPACRun : '+str(timer.interval))
    dump(modelPACRun, 'modelePAC_'+labelDataApprises+'.modele')
    
    with Timer() as timer:
        modelSGDCRunH.fit(X_RUN, y)
    print(str(datetime.now())+' - temps de traitement du fit de modelSGDCRunH : '+str(timer.interval))
    dump(modelSGDCRunH, 'modeleSGD_H_'+labelDataApprises+'.modele')
    with Timer() as timer:
        modelSGDCRunL.fit(X_RUN, y)
    print(str(datetime.now())+' - temps de traitement du fit de modelSGDCRunL : '+str(timer.interval))
    dump(modelSGDCRunL, 'modeleSGD_L_'+labelDataApprises+'.modele')
    with Timer() as timer:
        modelSGDCRunMH.fit(X_RUN, y)
    print(str(datetime.now())+' - temps de traitement du fit de modelSGDCRunMH : '+str(timer.interval))
    dump(modelSGDCRunMH, 'modeleSGD_MH_'+labelDataApprises+'.modele')
    
    with Timer() as timer:
        modelLSVCRunH.fit(X_RUN, y)
    print(str(datetime.now())+' - temps de traitement du fit de modelLSVCRunH : '+str(timer.interval))
    dump(modelLSVCRunH, 'modeleLSVC_H_'+labelDataApprises+'.modele')
    with Timer() as timer:
        modelLSVCRunSH.fit(X_RUN, y)
    print(str(datetime.now())+' - temps de traitement du fit de modelLSVCRunSH : '+str(timer.interval))
    dump(modelLSVCRunSH, 'modeleLSVC_SH_'+labelDataApprises+'.modele')
    
    return vectRun, modelLRRun, modelPACRun, modelSGDCRunH, modelSGDCRunL, modelSGDCRunMH, modelLSVCRunH, modelLSVCRunSH

def preditStream(vecteur, X, modelLR, modelPAC, modelSGDC_H, modelSGDC_L, modelSGDC_MH, modelLSVC_H, modelLSVC_SH):
    with Timer() as timer:
        X_to_predict = vecteur.transform(X)
    print(str(datetime.now())+' - temps de traitement du transform : '+str(timer.interval))
    
    with Timer() as timer:
        predictedLR = modelLR.predict(X_to_predict)
    print(str(datetime.now())+' - temps de prediction de modelLR : '+str(timer.interval))
    with Timer() as timer:
        predictedPAC = modelPAC.predict(X_to_predict)
    print(str(datetime.now())+' - temps de prediction de modelPAC : '+str(timer.interval))
    with Timer() as timer:
        predictedSGD_H = modelSGDC_H.predict(X_to_predict)
    print(str(datetime.now())+' - temps de prediction de modelSGDC_H : '+str(timer.interval))
    with Timer() as timer:
        predictedSGD_L = modelSGDC_L.predict(X_to_predict)
    print(str(datetime.now())+' - temps de prediction de modelSGDC_L : '+str(timer.interval))
    with Timer() as timer:
        predictedSGD_MH = modelSGDC_MH.predict(X_to_predict)
    print(str(datetime.now())+' - temps de prediction de modelSGDC_MH : '+str(timer.interval))
    with Timer() as timer:
        predictedSVC_H = modelLSVC_H.predict(X_to_predict)
    print(str(datetime.now())+' - temps de prediction de modelLSVC_H : '+str(timer.interval))
    with Timer() as timer:
        predictedSVC_SH = modelLSVC_SH.predict(X_to_predict)
    print(str(datetime.now())+' - temps de prediction de modelLSVC_SH : '+str(timer.interval))
    
    return predictedLR, predictedPAC, predictedSGD_H, predictedSGD_L, predictedSGD_MH, predictedSVC_H, predictedSVC_SH

In [10]:
dfLearnK = dfKhalil.copy()
dfToLearnK = dfKhalilToLearn.copy()

In [76]:
vectRunKhalil, modelLRRunKhalil, modelPACRunKhalil, modelSGDCRunHKhalil, modelSGDCRunLKhalil, modelSGDCRunMHKhalil, modelLSVCRunHKhalil, modelLSVCRunSHKhalil = \
    apprendModeles(dfLearnK["AcléEt"], dfLearnK["Acible"], 'Khalil')
vectRun1821, modelLRRun1821, modelPACRun1821, modelSGDCRunH1821, modelSGDCRunL1821, modelSGDCRunMH1821, modelLSVCRunH1821, modelLSVCRunSH1821 = \
    apprendModeles(df1821["AcléEt"], df1821["Acible"], '1821')

2021-07-30 03:23:21.075483 - __start__
2021-07-30 03:24:14.824428 - temps de traitement du fitting : 53.748944997787476
2021-07-30 03:25:01.586367 - temps de traitement du transform : 46.76193809509277
2021-07-30 03:51:02.171149 - temps de traitement du fit de modelLRRun : 1560.5766937732697
2021-07-30 03:54:40.288651 - temps de traitement du fit de modelPACRun : 217.61451292037964
2021-07-30 04:00:03.362518 - temps de traitement du fit de modelSGDCRunH : 322.6531181335449
2021-07-30 04:07:30.842201 - temps de traitement du fit de modelSGDCRunL : 447.0215787887573
2021-07-30 04:14:31.520306 - temps de traitement du fit de modelSGDCRunMH : 420.1962571144104


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


2021-07-30 04:51:33.840211 - temps de traitement du fit de modelLSVCRunH : 2221.8596115112305
2021-07-30 05:11:39.317186 - temps de traitement du fit de modelLSVCRunSH : 1205.0674242973328
2021-07-30 05:11:39.763985 - __start__
2021-07-30 05:12:56.869276 - temps de traitement du fitting : 77.10429191589355
2021-07-30 05:14:08.578266 - temps de traitement du transform : 71.7089900970459
2021-07-30 06:01:42.503777 - temps de traitement du fit de modelLRRun : 2853.9255101680756
2021-07-30 06:07:58.568654 - temps de traitement du fit de modelPACRun : 375.3502175807953
2021-07-30 06:16:33.739726 - temps de traitement du fit de modelSGDCRunH : 514.5278632640839
2021-07-30 06:28:00.061701 - temps de traitement du fit de modelSGDCRunL : 685.6810395717621
2021-07-30 06:38:34.212292 - temps de traitement du fit de modelSGDCRunMH : 633.4900217056274


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


2021-07-30 07:48:35.233721 - temps de traitement du fit de modelLSVCRunH : 4200.3694705963135
2021-07-30 08:24:59.382882 - temps de traitement du fit de modelLSVCRunSH : 2183.56108379364


In [17]:
vectRunKhalil = definiVect(dfLearnK["AcléEt"])
vectRun1821 = definiVect(df1821["AcléEt"])

2021-07-30 18:11:13.711772 - __start__
2021-07-30 18:12:06.078062 - temps de traitement du fitting : 52.36629009246826
2021-07-30 18:12:24.496151 - __start__
2021-07-30 18:13:42.289086 - temps de traitement du fitting : 77.79293513298035


### chargement des modèles enregistrés

In [12]:
modelLRRunKhalil = load('modeleLR_Khalil.modele')
modelPACRunKhalil = load('modelePAC_Khalil.modele')
modelSGDCRunHKhalil = load('modeleSGD_H_Khalil.modele')
modelSGDCRunLKhalil = load('modeleSGD_L_Khalil.modele')
modelSGDCRunMHKhalil = load('modeleSGD_MH_Khalil.modele')
modelLSVCRunHKhalil = load('modeleLSVC_H_Khalil.modele')
modelLSVCRunSHKhalil = load('modeleLSVC_SH_Khalil.modele')

modelLRRun1821 = load('modeleLR_1821.modele')
modelPACRun1821 = load('modelePAC_1821.modele')
modelSGDCRunH1821 = load('modeleSGD_H_1821.modele')
modelSGDCRunL1821 = load('modeleSGD_L_1821.modele')
modelSGDCRunMH1821 = load('modeleSGD_MH_1821.modele')
modelLSVCRunH1821 = load('modeleLSVC_H_1821.modele')
modelLSVCRunSH1821 = load('modeleLSVC_SH_1821.modele')

In [18]:
dfToLearnK["LR_predKhalil"], dfToLearnK["PAC_predKhalil"], dfToLearnK["SGD_H_predKhalil"], dfToLearnK["SGD_L_predKhalil"], dfToLearnK["SGD_MH_predKhalil"], dfToLearnK["SVC_H_predKhalil"], dfToLearnK["SVC_SH_predKhalil"] = \
    preditStream(vectRunKhalil, dfToLearnK["AcléEt"], modelLRRunKhalil, modelPACRunKhalil, modelSGDCRunHKhalil, modelSGDCRunLKhalil, modelSGDCRunMHKhalil, modelLSVCRunHKhalil, modelLSVCRunSHKhalil)
dfToLearnK["LR_pred1821"], dfToLearnK["PAC_pred1821"], dfToLearnK["SGD_H_pred1821"], dfToLearnK["SGD_L_pred1821"], dfToLearnK["SGD_MH_pred1821"], dfToLearnK["SVC_H_pred1821"], dfToLearnK["SVC_SH_pred1821"] = \
    preditStream(vectRun1821, dfToLearnK["AcléEt"], modelLRRun1821, modelPACRun1821, modelSGDCRunH1821, modelSGDCRunL1821, modelSGDCRunMH1821, modelLSVCRunH1821, modelLSVCRunSH1821)

2021-07-30 18:14:06.642367 - temps de traitement du transform : 0.7390725612640381
2021-07-30 18:14:07.105752 - temps de prediction de modelLR : 0.46338486671447754
2021-07-30 18:14:07.560190 - temps de prediction de modelPAC : 0.4544386863708496
2021-07-30 18:14:08.008172 - temps de prediction de modelSGDC_H : 0.4479813575744629
2021-07-30 18:14:08.462103 - temps de prediction de modelSGDC_L : 0.4539313316345215
2021-07-30 18:14:08.925619 - temps de prediction de modelSGDC_MH : 0.4635162353515625
2021-07-30 18:14:08.963374 - temps de prediction de modelLSVC_H : 0.03775453567504883
2021-07-30 18:14:08.994623 - temps de prediction de modelLSVC_SH : 0.031249284744262695
2021-07-30 18:14:09.727374 - temps de traitement du transform : 0.6793324947357178
2021-07-30 18:14:10.329140 - temps de prediction de modelLR : 0.6017656326293945
2021-07-30 18:14:10.928648 - temps de prediction de modelPAC : 0.5995082855224609
2021-07-30 18:14:11.534629 - temps de prediction de modelSGDC_H : 0.605980634

In [23]:
dfToLearnK2 = dfKhalilToLearn2.copy()

In [24]:
dfToLearnK2["LR_predKhalil"], dfToLearnK2["PAC_predKhalil"], dfToLearnK2["SGD_H_predKhalil"], dfToLearnK2["SGD_L_predKhalil"], dfToLearnK2["SGD_MH_predKhalil"], dfToLearnK2["SVC_H_predKhalil"], dfToLearnK2["SVC_SH_predKhalil"] = \
    preditStream(vectRunKhalil, dfToLearnK2["AcléEt"], modelLRRunKhalil, modelPACRunKhalil, modelSGDCRunHKhalil, modelSGDCRunLKhalil, modelSGDCRunMHKhalil, modelLSVCRunHKhalil, modelLSVCRunSHKhalil)
dfToLearnK2["LR_pred1821"], dfToLearnK2["PAC_pred1821"], dfToLearnK2["SGD_H_pred1821"], dfToLearnK2["SGD_L_pred1821"], dfToLearnK2["SGD_MH_pred1821"], dfToLearnK2["SVC_H_pred1821"], dfToLearnK2["SVC_SH_pred1821"] = \
    preditStream(vectRun1821, dfToLearnK2["AcléEt"], modelLRRun1821, modelPACRun1821, modelSGDCRunH1821, modelSGDCRunL1821, modelSGDCRunMH1821, modelLSVCRunH1821, modelLSVCRunSH1821)

2021-07-30 18:37:12.057460 - temps de traitement du transform : 1.9830901622772217
2021-07-30 18:37:12.612348 - temps de prediction de modelLR : 0.5548884868621826
2021-07-30 18:37:13.145167 - temps de prediction de modelPAC : 0.5328187942504883
2021-07-30 18:37:13.677521 - temps de prediction de modelSGDC_H : 0.5323536396026611
2021-07-30 18:37:14.216760 - temps de prediction de modelSGDC_L : 0.5392391681671143
2021-07-30 18:37:14.765151 - temps de prediction de modelSGDC_MH : 0.5483906269073486
2021-07-30 18:37:14.881051 - temps de prediction de modelLSVC_H : 0.11590003967285156
2021-07-30 18:37:14.998454 - temps de prediction de modelLSVC_SH : 0.11740303039550781
2021-07-30 18:37:17.139268 - temps de traitement du transform : 2.019845962524414
2021-07-30 18:37:18.042016 - temps de prediction de modelLR : 0.9027481079101562
2021-07-30 18:37:18.790317 - temps de prediction de modelPAC : 0.7483005523681641
2021-07-30 18:37:19.495671 - temps de prediction de modelSGDC_H : 0.705354213714

In [25]:
dfToLearnK2

,CdEnt,Entité,CodePays,CLE_COMPTE,Compte,Cd_Scat,Sous_catégorie_de_dépenses,Expense_Sub_Category,Cd_Cat,Catégorie_de_dépenses,...,SGD_MH_predKhalil,SVC_H_predKhalil,SVC_SH_predKhalil,LR_pred1821,PAC_pred1821,SGD_H_pred1821,SGD_L_pred1821,SGD_MH_pred1821,SVC_H_pred1821,SVC_SH_pred1821
0,DE22872,bnpp lease group cologne branch,DE,BPLG_DE.6391540000,personalbeschaffungskosten,spro0,z-Services professionnels - autres,z-Professional services - others,SPRO0,Z-Services professionnels - autres,...,professional services ; marcom ; events,professional services ; marcom ; events,professional services ; marcom ; events,professional services ; marcom ; events,professional services ; marcom ; events,professional services ; marcom ; events,professional services ; ps others ; -,professional services ; marcom ; events,professional services ; marcom ; events,professional services ; marcom ; events
1,DE22872,bnpp lease group cologne branch,DE,BPLG_DE.6391540000,personalbeschaffungskosten,spro0,z-Services professionnels - autres,z-Professional services - others,SPRO0,Z-Services professionnels - autres,...,professional services ; recruitment fees ; gen...,professional services ; recruitment fees ; gen...,professional services ; recruitment fees ; gen...,professional services ; marcom ; events,professional services ; ps others ; -,professional services ; marcom ; events,professional services ; ps others ; -,professional services ; recruitment fees ; gen...,professional services ; recruitment fees ; gen...,professional services ; recruitment fees ; gen...
2,DE22152,arval deutschland gmbh,DE,ARVAL_DE.XXX,verhan,???,Non affecté à une famille précise,Not affected to a precise family,???,ZZ-Non affecté à une famille précise,...,bnp paribas group ; - ; -,corporate services ; corporate real estate ; c...,corporate services ; corporate real estate ; c...,corporate services ; corporate real estate ; c...,corporate services ; corporate real estate ; c...,corporate services ; corporate real estate ; c...,bnp paribas group ; - ; -,corporate services ; corporate real estate ; c...,corporate services ; corporate real estate ; c...,corporate services ; corporate real estate ; c...
3,CN23534,bnpp (china) ltd,CN,SAP_LIGHT.SAP|676300,other - other,???,Non affecté à une famille précise,Not affected to a precise family,???,ZZ-Non affecté à une famille précise,...,technology ; software ; -,technology ; software ; -,technology ; software ; -,technology ; software ; -,technology ; software ; -,technology ; software ; -,technology ; software ; -,technology ; software ; -,technology ; software ; -,technology ; software ; -
4,DE22872,bnpp lease group cologne branch,DE,BPLG_DE.6391700000,sachversicherungen,assu0,z-Assurances - autres,z-Insurance - others,ASSU0,Z-Assurances - autres,...,corporate insurance ; - ; -,corporate insurance ; - ; -,corporate insurance ; - ; -,corporate insurance ; - ; -,corporate insurance ; - ; -,corporate insurance ; - ; -,corporate insurance ; - ; -,corporate insurance ; - ; -,corporate insurance ; - ; -,corporate insurance ; - ; -
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33264,CO44866,cardif colombia seguros generales sa,CO,CARDIF_CO.51909545,activos fijos menores,???,Non affecté à une famille précise,Not affected to a precise family,???,ZZ-Non affecté à une famille précise,...,corporate services ; corporate real estate ; c...,corporate services ; corporate real estate ; c...,corporate services ; corporate real estate ; c...,corporate services ; corporate real estate ; c...,corporate services ; corporate real estate ; c...,corporate services ; corporate real estate ; c...,corporate services ; corporate real estate ; c...,corporate services ; corporate real estate ; c...,corporate services ; corporate real estate ; c...,corporate services ; corporate real estate ; c...
33265,CO44866,cardif colombia seguros generales sa,CO,CARDIF_CO.5190959924,consultas desempleo experian,???,Non affecté à une famille

In [27]:
dfToLearnK2.to_excel('lignes à enrichir - streamées simple.xlsx')

In [21]:
def traiteRecursifDf(df, dico, nbLignes, nbrElemTotal, nbrParmiTotal, numCourant, numElem, numInitial, \
                     tableauVal, complementLabel):
    
    if numCourant == nbrParmiTotal:
        label = ''
        
        for i in range(0, len(tableauVal) - 1):
            label = label+str(tableauVal[i])
        
        labelPrec = label
        labelBase = str(tableauVal[0]) + str(tableauVal[len(tableauVal) - 1])
        label = label+str(tableauVal[len(tableauVal) - 1])
        
        #print(label)
        
        dfTemp = pd.DataFrame()
        if complementLabel == '':
            if nbrParmiTotal == 2:
                df[label] = [a if a==b else '-' for a,b in zip(df[tableauVal[0]],df[tableauVal[1]])]
            else:
                df[label] = [a if a==b else '-' for a,b in zip(df[labelBase],df[labelPrec])]
            
            if False:
                if nbrParmiTotal == 2:
                    df[label] = [1 if a==b else 0 for a,b in zip(df[tableauVal[0]],df[tableauVal[1]])]
                else:
                    df[label] = [a*b for a,b in zip(df[labelBase],df[labelPrec])]

                dico[label] = {}
                dico[label]['%'] = 100*sum(df[label])/nbLignes
        else:
            df[label+complementLabel]=[a*b for a,b in zip(df[str(numInitial)+complementLabel],df[label])]
            if sum(df[label]) > 0:
                dico[label]['%'+complementLabel] = 100*sum(df[label+complementLabel])/sum(df[label])
            else:
                dico[label]['%'+complementLabel] = 0
        
        if False:
            dico[label]['nbr'+complementLabel] = sum(df[label+complementLabel])

    else:
        for i in range(numElem+1, nbrElemTotal + 1):
            tableauVal.append(i)
            df, dico = traiteRecursifDf(df, dico, nbLignes, nbrElemTotal, nbrParmiTotal, numCourant+1, i, \
                                        numInitial, tableauVal, complementLabel)
            tableauVal.pop()
            
    return df, dico

def traiteDFRetourRec(dfTravail, champCible, ext):
    tableauLabels = ['LR_predKhalil', 'PAC_predKhalil',\
                    'SGD_H_predKhalil', 'SGD_L_predKhalil', 'SGD_MH_predKhalil',\
                    'SVC_H_predKhalil', 'SVC_SH_predKhalil',\
                    'LR_pred1821', 'PAC_pred1821',\
                    'SGD_H_pred1821', 'SGD_L_pred1821', 'SGD_MH_pred1821',\
                    'SVC_H_pred1821', 'SVC_SH_pred1821']
    #tableauLabels = ['LogisticRegression_liblinear_'+ext,'LogisticRegression_sag_'+ext,'LogisticRegression_saga_'+ext,\
    #                 'PassiveAggressive_'+ext,'SGD_hinge_'+ext,'SGD_log_'+ext,'SGD_modified_huber_'+ext,'LinearSVC_hinge_'+ext,\
    #                 'LinearSVC_squared_hinge_'+ext]
    nbTotalElem = len(tableauLabels)
    for i,label in zip([x for x in range(1, nbTotalElem + 1)],tableauLabels):
        dfTravail[i]=dfTravail[label]
    
    nbLignes = dfTravail.shape[0]
    
    dicoTemp={}
    if False:
        dicoTemp['nbL'] = {}
        dicoTemp['nbL']['nbr'] = nbLignes
        dicoTemp['nbL']['%'] = 0
        dicoTemp['nbL']['nbr OK'] = 0
        dicoTemp['nbL']['% OK'] = 0

        #si chacun est pareil que cible, résultat attendu
        for i in range(1, nbTotalElem + 1):
            dfTravail[str(i)+' OK'] = [1 if a==b else 0 for a,b in zip(dfTravail[i],dfTravail[champCible])]
            dicoTemp[str(i)] = {}
            dicoTemp[str(i)]['nbr'] = nbLignes
            dicoTemp[str(i)]['%'] = 100
            dicoTemp[str(i)]['nbr OK'] = sum(dfTravail[str(i)+' OK'])
            dicoTemp[str(i)]['% OK'] = 100*sum(dfTravail[str(i)+' OK'])/nbLignes
    
    #lance le travail récursif
    #considère nbrElemPrisEnCompte parmi le nombre total d'éléments
    #soit 2 parmi n, puis 3 parmi n,...
    for nbrElemPrisEnCompte in range(2, len(tableauLabels) + 1):
        for i in range(1, len(tableauLabels) + 1):
            dfTravail, dicoTemp = traiteRecursifDf(dfTravail, dicoTemp, nbLignes, len(tableauLabels), \
                                                   nbrElemPrisEnCompte, 1, i, i, [i], '')
            if False:
                dfTravail, dicoTemp = traiteRecursifDf(dfTravail, dicoTemp, nbLignes, len(tableauLabels), \
                                                       nbrElemPrisEnCompte, 1, i, i, [i], ' OK')
    
    dfResultatsT = pd.DataFrame.from_dict(dicoTemp)
    
    return dfTravail, dfResultatsT

In [89]:
with Timer() as timerTout:
    dfCompletRec, dfResultatsTRec = traiteDFRetourRec(dfToLearnK, '', '')
    #a=1
print()
print(str(datetime.now())+' - temps de generation des possibles : '+str(timerTout.interval))


2021-07-30 17:00:50.542904 - temps de generation des possibles : 625.9604425430298


In [93]:
dfCompletRec

,CdEnt,Entité,CodePays,CLE_COMPTE,Compte,Cd_Scat,Sous_catégorie_de_dépenses,Expense_Sub_Category,Cd_Cat,Catégorie_de_dépenses,...,123456781011121314,123456791011121314,123456891011121314,123457891011121314,123467891011121314,123567891011121314,124567891011121314,134567891011121314,234567891011121314,1234567891011121314
0,AE002,bnpp abu dhabi branch,AE,SAP.625110,security - outsourced manpower,garde,Gardiennage,Security,CHARG,Charges,...,corporate services ; corporate real estate ; c...,corporate services ; corporate real estate ; c...,corporate services ; corporate real estate ; c...,corporate services ; corporate real estate ; c...,corporate services ; corporate real estate ; c...,corporate services ; corporate real estate ; c...,corporate services ; corporate real estate ; c...,corporate services ; corporate real estate ; c...,corporate services ; corporate real estate ; c...,corporate services ; corporate real estate ; c...
1,AE001,bnpp dubai branch,AE,SAP.603425,club subscriptions,abonn,Abonnements professionnels,Professional subscriptions,COTIS,Cotisations et abonnements,...,-,-,-,-,-,-,-,-,-,-
2,AE002,bnpp abu dhabi branch,AE,SAP.626205,upkeep operating premises,main0,z-Coûts de maintenance - autres,z-Costs of maintenance - others,MAINT,Coûts de maintenance,...,corporate services ; corporate real estate ; m...,corporate services ; corporate real estate ; m...,corporate services ; corporate real estate ; m...,corporate services ; corporate real estate ; m...,corporate services ; corporate real estate ; m...,corporate services ; corporate real estate ; m...,corporate services ; corporate real estate ; m...,corporate services ; corporate real estate ; m...,corporate services ; corporate real estate ; m...,corporate services ; corporate real estate ; m...
3,AE002,bnpp abu dhabi branch,AE,SAP.604240,depn - vehicle,???,Non affecté à une famille précise,Not affected to a precise family,???,ZZ-Non affecté à une famille précise,...,banking services ; credit cards (incl processi...,banking services ; credit cards (incl processi...,banking services ; credit cards (incl processi...,banking services ; credit cards (incl processi...,banking services ; credit cards (incl processi...,banking services ; credit cards (incl processi...,banking services ; credit cards (incl processi...,banking services ; credit cards (incl processi...,banking services ; credit cards (incl processi...,banking services ; credit cards (incl processi...
4,AE002,bnpp abu dhabi branch,AE,SAP.603427,other administrative expenses,resto,Restaurants,Restaurants,RESTO,Restaurants,...,-,-,-,-,-,-,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12745,SE65826,sevendays,-,MSDYNAX_EB.835090,sundry building costs,gtrv0,z-Gros oeuvre - autres,z-Masonry - others,GTRVX,Gros oeuvre,...,corporate services ; corporate real estate ; f...,corporate services ; corporate real estate ; f...,corporate services ; corporate real estate ; f...,corporate services ; corporate real estate ; f...,corporate services ; corporate real estate ; f...,corporate services ; corporate real estate ; f...,corporate services ; corporate real estate ; f...,corporate services ; corporate real estate ; f...,corporate services ; corporate real estate ; f...,corporate services ; corporate real estate ; f...
12746,SE65826,sevendays,-,MSDYNAX_EB.880010,other costs,char0,z-Charges - autres,z-Charges - others,CHARG,Charges,...,-,-,-,-,-,-,-,-,-,-
12747,SE65826,sevendays,-,MSDYNAX_EB.880010,other costs,char0,z-Charges - autres,z-Charges - others,CHARG,Charges,...,corporate services ; corporate real estate ; c...,corporate services ; corporate real estate ; c...,corporate services ; corporate real estate ; c...,corporate services ; corporate real estate ; c...,corporate services ; corporate real estate ; c...,corporate services ; corporate real estate ; c...,corporate services ; corporate real estate ; c...,corporate services ; corporate real estate ; c...,corporate s

In [35]:
dfToLearnK

,CdEnt,Entité,CodePays,CLE_COMPTE,Compte,Cd_Scat,Sous_catégorie_de_dépenses,Expense_Sub_Category,Cd_Cat,Catégorie_de_dépenses,...,SGD_MH_predKhalil,SVC_H_predKhalil,SVC_SH_predKhalil,LR_pred1821,PAC_pred1821,SGD_H_pred1821,SGD_L_pred1821,SGD_MH_pred1821,SVC_H_pred1821,SVC_SH_pred1821
0,AE002,bnpp abu dhabi branch,AE,SAP.625110,security - outsourced manpower,garde,Gardiennage,Security,CHARG,Charges,...,corporate services ; corporate real estate ; c...,corporate services ; corporate real estate ; c...,corporate services ; corporate real estate ; c...,corporate services ; corporate real estate ; c...,corporate services ; corporate real estate ; c...,corporate services ; corporate real estate ; c...,corporate services ; corporate real estate ; c...,corporate services ; corporate real estate ; c...,corporate services ; corporate real estate ; c...,corporate services ; corporate real estate ; c...
1,AE001,bnpp dubai branch,AE,SAP.603425,club subscriptions,abonn,Abonnements professionnels,Professional subscriptions,COTIS,Cotisations et abonnements,...,corporate services ; general services ; social...,corporate services ; general services ; social...,corporate services ; general services ; social...,market data ; - ; -,market data ; - ; -,market data ; - ; -,market data ; - ; -,market data ; - ; -,market data ; - ; -,market data ; - ; -
2,AE002,bnpp abu dhabi branch,AE,SAP.626205,upkeep operating premises,main0,z-Coûts de maintenance - autres,z-Costs of maintenance - others,MAINT,Coûts de maintenance,...,corporate services ; corporate real estate ; m...,corporate services ; corporate real estate ; m...,corporate services ; corporate real estate ; m...,corporate services ; corporate real estate ; m...,corporate services ; corporate real estate ; m...,corporate services ; corporate real estate ; m...,corporate services ; corporate real estate ; m...,corporate services ; corporate real estate ; m...,corporate services ; corporate real estate ; m...,corporate services ; corporate real estate ; m...
3,AE002,bnpp abu dhabi branch,AE,SAP.604240,depn - vehicle,???,Non affecté à une famille précise,Not affected to a precise family,???,ZZ-Non affecté à une famille précise,...,banking services ; credit cards (incl processi...,banking services ; credit cards (incl processi...,banking services ; credit cards (incl processi...,banking services ; credit cards (incl processi...,banking services ; credit cards (incl processi...,banking services ; credit cards (incl processi...,banking services ; credit cards (incl processi...,banking services ; credit cards (incl processi...,banking services ; credit cards (incl processi...,banking services ; credit cards (incl processi...
4,AE002,bnpp abu dhabi branch,AE,SAP.603427,other administrative expenses,resto,Restaurants,Restaurants,RESTO,Restaurants,...,corporate services ; travel ; restaurants,professional services ; auditors ; -,corporate services ; travel ; restaurants,corporate services ; travel ; travel others (n...,corporate services ; travel ; travel others (n...,corporate services ; travel ; travel others (n...,corporate services ; travel ; travel others (n...,corporate services ; travel ; travel others (n...,corporate services ; travel ; travel others (n...,corporate services ; travel ; travel others (n...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12745,SE65826,sevendays,-,MSDYNAX_EB.835090,sundry building costs,gtrv0,z-Gros oeuvre - autres,z-Masonry - others,GTRVX,Gros oeuvre,...,corporate services ; corporate real estate ; f...,corporate services ; corporate real estate ; f...,corporate services ; corporate real estate ; f...,corporate services ; corporate real estate ; f...,corporate services ; corporate real estate ; f...,corporate services ; corporate real estate ; f...,corporate services ; corporate real estate ; f...,corporate services ; corporate real estate ; f...,corporate services ; corporate real estate ; f...,corporate services ; corporate r

In [42]:
def streamsIdentiques(df):
    dfToLearnK['14']=[a if a==b else "-" for a,b in zip(dfToLearnK.iloc[:,34+i])]
    for i in range(0,14):
        dfToLearnK.iloc[:,34+i]

0        corporate services ; corporate real estate ; c...
1        corporate services ; general services ; social...
2        corporate services ; corporate real estate ; m...
3        banking services ; credit cards (incl processi...
4                corporate services ; travel ; restaurants
                               ...                        
12745    corporate services ; corporate real estate ; f...
12746    corporate services ; corporate real estate ; c...
12747    corporate services ; corporate real estate ; c...
12748      corporate services ; general services ; postage
12749    corporate services ; corporate real estate ; c...
Name: LR_predKhalil, Length: 12750, dtype: object
0        corporate services ; corporate real estate ; c...
1        corporate services ; general services ; social...
2        corporate services ; corporate real estate ; m...
3        banking services ; credit cards (incl processi...
4                corporate services ; travel ; restaurants
      